In [6]:
from pathlib import Path
from pystk2_gymnasium import AgentSpec
from functools import partial
import torch
import inspect
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from bbrl.agents import Agents, TemporalAgent
from bbrl.workspace import Workspace
from bbrl.agents import Agent
import gymnasium as gym


import sys
import os


# Note the use of relative imports
from actors import Actor
from pystk_actor import env_name, get_wrappers, player_name

In [42]:
class SamplingActor(Agent):
    """Samples random actions"""

    def __init__(self, action_space: gym.Space):
        super().__init__()
        self.action_space = action_space

    def forward(self, t: int):
        self.set(("action", t), torch.tensor([[4, 1, 0, 1, 1, 0, 6]]))#torch.LongTensor([self.action_space.sample()]))

In [43]:
env_name = "supertuxkart/flattened_multidiscrete-v0"
# env_name = "supertuxkart/full-v0"

# Setup the environment
make_stkenv = partial(
    make_env,
    env_name,
    wrappers=get_wrappers(),
    render_mode="human", # human for video, else None 
    autoreset=True,
    agent=AgentSpec(use_ai=False, name=player_name), # use_ai=False for using the "action" line of the workspace
)

env_agent = ParallelGymAgent(make_stkenv, 1)
env = env_agent.envs[0]

# (2) Learn

actor = SamplingActor(env.action_space)
temporal_agent = TemporalAgent(Agents(env_agent, actor))

workspace = Workspace()
temporal_agent(workspace, t=0, n_steps=200)

# (3) Save the actor state
mod_path = Path(inspect.getfile(get_wrappers)).parent
torch.save(actor.state_dict(), mod_path / "pystk_actor.pth")


..:: Antarctica Rendering Engine 2.0 ::..


In [59]:
env.step()

TypeError: step() missing 1 required positional argument: 'action'

In [39]:
# for key in workspace.keys():
#     print(key, workspace([key])
workspace["action"]

tensor([[[4, 1, 0,  ..., 1, 0, 6]],

        [[3, 1, 0,  ..., 1, 1, 3]],

        [[1, 0, 1,  ..., 0, 1, 5]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 1, 1]],

        [[1, 1, 1,  ..., 1, 0, 4]]])